In [1]:
from redis import Redis
from dotenv import load_dotenv
import os
from dm.game import Game
import asyncio

In [2]:
load_dotenv()
REDIS_HOST = os.getenv('REDIS_HOST')
REDIS_PORT = os.getenv('REDIS_PORT')
REDIS_USERNAME = os.getenv('REDIS_USERNAME')
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD')
API_HOST = os.getenv('API_HOST')
DUNGEON_MASTER_TOKEN = os.getenv('DUNGEON_MASTER_TOKEN')

# Connect to redis
redis_client = Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD, db=0, protocol=3, decode_responses=True)

# Create game client
game = Game(redis_client, 2, dm_token=DUNGEON_MASTER_TOKEN, api_host=API_HOST)

## Aggregate relevant players and monsters by plots

1. Get all players from redis
2. Collect all unique geohashes from players
3. Segment out plots for AI to work on, a plot contains a set of players, monsters in a geohash
4. There should be no overlaps in the plots (players, monsters) so that the AI can run in parallel

In [3]:

num_processed = 0
process_limit = 10 # for testing

for player in game.logged_in_players():
    player_geohash = player['loc'][0]
    for monster in game.get_nearby_entities(player_geohash, monsters=True):
        print(monster['monster'])
        num_processed +=1
        if num_processed > process_limit:
            break
    if num_processed > process_limit:
            break
        

monster_goblin8807747
monster_goblin8813432
monster_giantSpider8828793
monster_dragon8781845
monster_goblin8797364
monster_dragon8835705
monster_goblin8849321
monster_goblin8865341
monster_giantSpider8873163
monster_goblin8856152
monster_dragon8835705


In [4]:
m = game.fetch_entity('monster_goblin8856152')
print(m)
m['skills']

{'monster': 'monster_goblin8856152', 'name': 'goblin', 'beast': 'goblin', 'loc': ['h9h8pt9w'], 'locT': 'geohash', 'locI': '@', 'hp': 10, 'mnd': 1, 'cha': 1, 'lum': 0, 'umb': 0, 'skills': {'beast': 1, 'dirtyfighting': 1, 'monster': 1, 'firstaid': 1}, 'buclk': 1728280174225, 'buf': [], 'dbuf': [], 'pthclk': 1728280174125, 'pthdur': 100, 'pth': ['s'], 'pthst': 'h9h8pt9x'}


{'beast': 1, 'dirtyfighting': 1, 'monster': 1, 'firstaid': 1}

In [5]:
await game.perform_monster_move(m, directions=['s'])

In [6]:
await game.query_monster_abilities(m)

['eyePoke', 'bandage']